In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display
from sklearn.cluster import DBSCAN
import plotly.graph_objects as go
from scipy.signal import butter, filtfilt
from sklearn.neighbors import NearestNeighbors

In [148]:
# Path to the CSV file
file_path = '/home/ubuntu/MoCap/T2_3A_thirdcomp_failed.csv'

# Read the file and find the start of frame data
with open(file_path, 'r') as file:
    lines = file.readlines()

frame_start_index = None
for index, line in enumerate(lines):
    if line.startswith('frame'):
        frame_start_index = index
        break



In [149]:
all_positions = []  # This will store a list of positions for each frame
time_stamps = []
numb_points= []
# Assuming `lines` and `frame_start_index` have been defined as per your previous snippets.
for line in lines[frame_start_index:]:
    parts = line.split(',')
    if parts[0] != 'frame':
        continue
    
    frame_time = float(parts[2])
    time_stamps.append(frame_time)
    
    num_points = int(parts[4])

    frame_positions = []  # Store positions for this frame only
    numb_points.append(num_points)
    for i in range(num_points):
        base_index = 5 + i * 5
        x = float(parts[base_index])
        y = float(parts[base_index + 1])
        z = float(parts[base_index + 2])
        frame_positions.append((x, y, z))
    
    all_positions.append(frame_positions)  # Append list of tuples for each frame

In [150]:
# Assuming all_positions is a list of lists where each sub-list contains tuples (x, y, z)
# Flatten all_points for plotting
all_points = [point for frame in all_positions for point in frame]
all_points = np.array(all_points)  # Convert to a NumPy array for easier handling

# Check if there are points to plot
if all_points.size == 0:
    print("No points to plot.")
else:
    # Create the 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=all_points[:, 0],
        y=all_points[:, 1],
        z=all_points[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color='blue',  # Color of the markers
        )
    )])

    # Update plot layout
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            xaxis_title='X Coordinate',
            yaxis_title='Y Coordinate',
            zaxis_title='Z Coordinate'
        ),
        title='All Marker Positions Across All Frames'
    )

    # Show the plot
plot2_path = "all_points.html"
fig.write_html(plot2_path)

In [151]:
# # Initialize containers for the filtered points and outliers
# filtered_positions = []
# outlier_positions = []
# filtered_time_stamps = []

# # Process each frame for outlier detection
# for frame, time_stamp in zip(all_positions, time_stamps):
#     if len(frame) > 0:
#         points = np.array(frame)  # Convert frame to a numpy array
        
#         # Calculate Q1, Q3, and IQR
#         Q1 = np.percentile(points, 25, axis=0)
#         Q3 = np.percentile(points, 75, axis=0)
#         IQR = Q3 - Q1
        
#         # Define lower and upper bounds for outliers
#         lower_bound = Q1 - 2 * IQR
#         upper_bound = Q3 + 2 * IQR
        
#         # Determine inliers and outliers
#         is_inlier = (points >= lower_bound) & (points <= upper_bound)
#         inliers = points[np.all(is_inlier, axis=1)]
#         outliers = points[~np.all(is_inlier, axis=1)]

#         # Append the results to their respective lists
#         if inliers.size > 0:
#             filtered_positions.append(inliers.tolist())
#             filtered_time_stamps.append(time_stamp)
#         if outliers.size > 0:
#             outlier_positions.append(outliers.tolist())


In [153]:

# Fit the KNN model
k = 5
nbrs = NearestNeighbors(n_neighbors=k).fit(all_points)
distances, _ = nbrs.kneighbors(all_points)

# Calculate the average distance to k neighbors
avg_distances = distances.mean(axis=1)

# Define a threshold for identifying outliers
threshold = np.mean(avg_distances) + 2 * np.std(avg_distances)

# Determine inliers and outliers
inliers = all_points[avg_distances <= threshold]
outliers = all_points[avg_distances > threshold]

# Reconstruct the frames with filtered points
filtered_positions = []
filtered_time_stamps = []
outlier_positions = []

current_index = 0
for frame, time_stamp in zip(all_positions, time_stamps):
    frame_length = len(frame)
    frame_points = all_points[current_index:current_index + frame_length]
    frame_avg_distances = avg_distances[current_index:current_index + frame_length]
    
    frame_inliers = frame_points[frame_avg_distances <= threshold]
    frame_outliers = frame_points[frame_avg_distances > threshold]
    
    if len(frame_inliers) > 0:
        filtered_positions.append(frame_inliers)
        filtered_time_stamps.append(time_stamp)
    
    if len(frame_outliers) > 0:
        outlier_positions.append(frame_outliers)
    
    current_index += frame_length

# Convert lists back to numpy arrays if needed
filtered_positions = np.array(filtered_positions)
filtered_time_stamps = np.array(filtered_time_stamps)
outlier_positions = np.array(outlier_positions)


In [154]:
# Assuming 'filtered_positions' and 'outlier_positions' contain lists of 3D coordinates for each frame
# Flatten the lists of lists into single lists for Plotly plotting
filtered_points = np.vstack(filtered_positions)  # Stacking all filtered points
outlier_points = np.vstack(outlier_positions)  # Stacking all outlier points

# Create a Plotly graph object for the interactive plot
fig = go.Figure()

# Add the filtered points in blue
fig.add_trace(go.Scatter3d(
    x=filtered_points[:, 0],
    y=filtered_points[:, 1],
    z=filtered_points[:, 2],
    mode='markers',
    marker=dict(
        size=3,
        color='blue',  # Use blue to indicate filtered points
        opacity=0.8
    ),
    name='Filtered Points'
))

# Add the outliers in red
fig.add_trace(go.Scatter3d(
    x=outlier_points[:, 0],
    y=outlier_points[:, 1],
    z=outlier_points[:, 2],
    mode='markers',
    marker=dict(
        size=3,
        color='red',  # Use red to indicate outliers
        opacity=0.8
    ),
    name='Outliers'
))

# Update the layout to create a visually appealing and informative plot
fig.update_layout(
    title='Interactive 3D Scatter Plot of MoCap Data',
    scene=dict(
        xaxis_title='X Coordinates',
        yaxis_title='Y Coordinates',
        zaxis_title='Z Coordinates'
    ),
    legend_title_text='Point Type'
)

# Show the plot

plot3_path = "outlyers_detection.html"
fig.write_html(plot3_path)

In [155]:

filtered_avg_positions = []
# Assume 'all_positions' is a list of numpy arrays, where each array contains points for a frame
for frame in all_positions:
    if len(frame) > 0:  # Ensure there are points in the frame
        points = np.array(frame)
        average_position = np.mean(points, axis=0)  # Compute the mean along each column (x, y, z)
        filtered_avg_positions.append(average_position.tolist())
    else:
        filtered_avg_positions.append([np.nan, np.nan, np.nan])  # Append NaNs if frame is empty


In [156]:
# Define a function to apply a Butterworth low-pass filter
def butter_lowpass_filter(data, cutoff, fs, order=5):
    nyq = 0.5 * fs  # Nyquist Frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data, axis=0)
    return y

# Sample frequency and cutoff frequency
fs = 60  # Sample rate, Hz (adjust based on your data sampling rate)
cutoff = 3  # Desired cutoff frequency of the filter, Hz

# Apply the filter
filtered_avg_positions = butter_lowpass_filter(filtered_avg_positions, cutoff, fs, order=5)

In [157]:


# Convert list of average positions to a NumPy array for easier handling
filtered_avg_positions = np.array(filtered_avg_positions)

# Create a Plotly figure

fig = go.Figure(data=[go.Scatter3d(
    x=filtered_avg_positions[:, 0],
    y=filtered_avg_positions[:, 1],
    z=filtered_avg_positions[:, 2],
    mode='lines+markers',

    text=[f'x: {round(xp,4)}, y: {round(yp,4)}, z: {round(zp,4)}, t: {t}, n of points: {np}' for np, xp, yp, zp, t in zip(numb_points, filtered_avg_positions[:, 0], filtered_avg_positions[:, 1], filtered_avg_positions[:, 2], filtered_time_stamps)],
    hoverinfo='text',
    marker=dict(
        size=5,
        color=np.linspace(0, filtered_time_stamps[-1], len(filtered_time_stamps)),  # Gradient color based on time
        colorscale='Viridis',
        showscale=True
    ),
    
    line=dict(
        color='#1f77b4',  # Consistent color for the line
        width=2
    )
)])


# Update plot layout for better visualization
fig.update_layout(
    title='Trajectory of Average Positions Over Time',
    scene=dict(
        xaxis_title='X Coordinates',
        yaxis_title='Y Coordinates',
        zaxis_title='Z Coordinates'
    ),
    legend_title_text='Frame'
)

# Show the interactive plot

plot4_path = "avarageCM_filtered.html"
fig.write_html(plot4_path)